## [Chapter 3] Controlling Relevance

In the [last notebook](1.vectors-and-text-similarity.ipynb), we introduced how queries and documents can be represented as vectors, how cosine similarity can be used as a relevance function to compare queries and documents, and how tf-idf ranking can be used to create a feature weight that balances both strength of occurrence (TF) and significance of a term (IDF) for each term in a term-based vector.

In this notebook, we will show how a full relevance function can be specified and controlled in a search engine (Apache Solr). Let's start with showing off the default similarity calculation leveraged by all Lucene-based search engines: BM25

### BM25 (Best Match Okapi 25)
The BM25 algorithm is the default similarity function in Apache Lucene, Apache Solr, Elasticsearch, Lucidworks Fusion, and other Lucene-based search engines. The algorithm leverages TF and IDF, but also incorporates many additional configurable options. The full BM25 calculation is found in Figure 3.7 in the book.

Instead of reimplementing the full BM25 algorithm in Python, let's go ahead and switch over to using our search engine (Apache Solr) and see how it performs the calculation.

### Creating our Collection

### Listing 3.9

In [1]:
from aips import set_engine, get_engine

set_engine("weaviate")
engine = get_engine()
print(type(engine))

<class 'engines.weaviate.WeaviateEngine.WeaviateEngine'>


In [4]:
from graphql_query import Query, Argument, Field, Operation, Directive
from engines.weaviate.config import WEAVIATE_HOST, WEAVIATE_PORT, WEAVIATE_URL
import graphql_query
import requests

query = "\"ipod\""
args = [Argument(name="limit", value=10),
        Argument(name="bm25", value=[Argument(name="query", value=query)])]
score = Field(name="_additional", fields=["score"])
request = Operation(type="Get", queries=[Query(name="Products", arguments=args,
                                               fields=["upc", "name", "short_description",
                                                       "long_description", score])])
gql_query = "{" + request.render() + "}"
request = {"query": gql_query}

print(request)
response = requests.post(f"{WEAVIATE_URL}/v1/graphql", json=request).json()
display(response)

{'query': '{Get {\n  Products(\n    limit: 10\n    bm25: {\n      query: "ipod"\n    }\n  ) {\n    upc\n    name\n    short_description\n    long_description\n    _additional {\n      score\n    }\n  }\n}}'}


{'data': {'Get': {'Products': [{'_additional': {'score': '1.5809941'},
     'long_description': 'This USB power supply is compatible with many USB-enabled digital music players and is ideal for travel. The folding swing plug can be stored easily.',
     'name': 'Sony - USB Power Supply',
     'short_description': 'Compatible with Apple&#xAE; iPod&#xAE; nano, iPod classic and 4th- and 5th-generation iPod digital music players; for travel use; folding swing plug',
     'upc': '27242777996'},
    {'_additional': {'score': '1.5565579'},
     'long_description': 'Enjoy the media on your iPod on the big screen with this composite A/V cable that lets you view photos and video clips on your TV.51.99',
     'name': 'Apple&#xAE; - Composite A/V Cable for Apple&#xAE; iPod&#x2122;',
     'short_description': 'Compatible with 3rd-generation iPod nano, iPod photo, iPod video, iPod classic and iPod touch; audio and USB connectivity options51.99',
     'upc': '885909271139'},
    {'_additional': {'sco

In [3]:
engine = get_engine()
print(type(engine))
engine.create_collection("products", log=True) #Upper cases collection name?

<class 'engines.weaviate.WeaviateEngine.WeaviateEngine'>
Wiping "products" collection
Creating "products" collection
Schema: {
  "class": "products",
  "properties": [
    {
      "name": "upc",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "_text_",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "name_ngram",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "name_fuzzy",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "short_description_ngram",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "name",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "short_description",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "long_description",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "manufacturer",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "has_promotion",
      "dataType": [
        

In [4]:
name = "Sphere" 
#!pip install requests
import requests
requests.delete(f"http://aips-weaviate:8090/v1/schema/Products")

<Response [200]>

In [5]:
from aips import *

engine = get_engine()
collection = engine.create_collection("cat_in_the_hat", log=True)

Wiping "cat_in_the_hat" collection
Creating "cat_in_the_hat" collection
Schema: {
  "class": "cat_in_the_hat",
  "properties": [
    {
      "name": "kid",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "title",
      "dataType": [
        "text"
      ]
    },
    {
      "name": "description",
      "dataType": [
        "text"
      ]
    }
  ]
}
Status: <Response [200]>
Response: <Response [200]>


### Add some documents

### Listing 3.10

In [6]:
docs = [
    {
        "kid": "doc1",
        "title": "Worst",
        "description": "The interesting thing is that the person in the wrong made the right decision in the end."
    },
    {
        "kid": "doc2",
        "title": "Best",
        "description": "My favorite book is the cat in the hat, which is about a crazy cat who breaks into a house and creates a crazy afternoon for two kids."
        
    },
    {
        "kid": "doc3",
        "title": "Okay",
        "description": "My neighbors let the stray cat stay in their garage, which resulted in my favorite hat that I let them borrow being ruined."        
    }
]
collection = engine.get_collection("cat_in_the_hat")
response = collection.add_documents(docs)
engine.print_status(response)

Successfully written 3 documents


### Inspecting BM25 Score
Now, let's execute a query for `the cat in the hat`, and see how each document scores using the BM25 similarity calculation:

### Listing 3.11

In [7]:
query = "the cat in the hat"
request = {"query": query,
           "query_fields": ["description"],
           "return_fields": ["id", "title", "description", "score"],
           "explain": True}

response = collection.search(**request)

display_search(query, response["docs"])

query Get {
  products(
    limit: 10
    bm25: {
      query: some query
    }
  ) {
    id
    title
    description
    _additional {
      score
    }
  }
}


NameError: name 'WEAVIATE_URL' is not defined

While the BM25 calculation is much more complex than the TF-IDF feature weight calculations we saw in the [last notebook](ch3-vectors-and-text-similarity.ipynb), it is fundamentally still derived from TF-IDF at it's core. As such, you'll notice that the search results actually return in the exact same order as our TF-IDF calculations from the last notebook:

<pre>
doc2: 0.6823196
doc3: 0.6285005
doc1: 0.3132525
</pre>

Our query for *the cat in the hat* can still very much be thought of as a vector of the BM25 scores for each of the terms: ["the", "cat", "in", "the", "hat"].

What may not be obvious, however, is that the feature weights for each of these features are actually just overridable functions. For example, this query could alternatively be expressed as the vector:

<pre>
[ query("the"), query("cat"), query("in"), query("the"), query("hat") ]
</pre>

In Solr syntax, this would be:

<pre>
{"query": '{!func}query("the") {!func}query("cat") {!func}query("in") {!func}query("the") {!func}query("hat")'}
</pre>

For that query, we get the following:

### Listing 3.12

In [ ]:
query = '{!func}query("the") {!func}query("cat") {!func}query("in") {!func}query("the") {!func}query("hat")'
request = {"query": query,
           "return_fields": ["id", "title", "score"],
           "query_fields": "description"}

response = collection.search(**request)

display_search(query, response["docs"])

The scores are exactly the same! Not only that, but once we realize that every term in a query is nothing more than a configurable scoring function, it opens up tremendous possibilities for manipulating that scoring function.

### Functions, Functions, Everywhere!
Now that we've seen that the relevance score for each term in our queries is simply a function operating on that term to generate a feature weight, the next logical question is "what OTHER kinds of functions can I use in my queries?".

We've already encountered the *query* function, which is effectively the default calculation that executes whenever no explicit function is specified, and which uses the BM25 similarity algorithm by default.

But what if we want to consider some other features in our scoring calculation, perhaps some that are not text-based?

Here is a partial list of common relevance techniques:
- Geospatial Boosting (documents near the user running the query should rank higher)
- Date Boosting (Newer documents should get a higher relevancy boost)
- Popularity Boosting (Documents which are more popular should get a higher relevancy boost)
- Field Boosting (Terms matching in certain fields should get a higher weight than in other fields)
- Category Boosting (Documents in categories related to query terms should get a higher relevancy boost)
- Phrase Boosting (documents matching multi-term phrases in the query should rank higher than those only matching the words separately)
- ...

Many of these techniques are built into specific query parsers in Solr, either through query syntax or through query parser options. For example, field boosting can be accomplished through the `qf` parameter on the `edismax` query parser:

  <pre>
  {"query": "the cat in the hat",
   <b>"fields": ["title^10", "description^2.5"]</b>,
   <b>"query_parser": "edismax"</b>}  

  {"query": "the cat in the hat",
   "params": {"defType": "edismax",
              "qf": "title^10 description^2.5"}}
  </pre>

Boosting on full phrase matching, on two-word phrases, and on three-word phrases is also a native feature of the edismax query parser:

- Boost docs containing the exact phrase "the cat in the hat" in the `title` field:

  <pre>
  {"query": "the cat in the hat",
   "params": {"defType": "edismax",
              "qf": "title description",
              <b>"pf": "title"</b>}}
  </pre>

- Boost docs containing "the cat", "cat in", "in the", or "the hat" in the `title` or `description` field:

  <pre>
  {"query": "the cat in the hat",
   "params": {"defType": "edismax",
              "qf": "title description",
              <b>"pf2": "title description"</b>}}
  </pre>

- Boost docs containing "the cat in" or "in the hat" in the `description` field:

  <pre>
  {"query": "the cat in the hat",
   "params": {"defType": "edismax",
              "qf": "title description",
              <b>"pf3": "description"</b>}}
  </pre>


Many of the relevancy boosting techniques will require constructing your own features leveraging function queries, however. For example, if we wanted to create a query that did nothing more than boost the relevance ranking of documents geographically closest to the user running the search (relevance based on distance away), we could issue the following query:

  <pre>
  {"query": "*",
   "sort": "<b>geodist(location, $user_latitude, $user_longitude) asc</b>",
   "params": {"user_latitude": 33.748,
              "user_longitude": -84.39}}
  </pre>

The above is using the `sort` parameter to strictly order documents by the calculated value from the `geodist` function. This works great if we want to order results by a single feature, but what if we want to construct a more nuanced sort based upon multiple features? To accomplish this, we will update our query to include each of these function in each document's relevance calculation, and then sort by the relevance score:

  <pre>
  {"query": <b>"{!func}scale(query($keywords),0,25) 
                {!func}recip(geodist($lat_long_field,$user_latitude,$user_longitude),1,25,1)
                {!func}recip(ms(NOW/HOUR,modify_date),3.16e-11,25,1)
                {!func}scale(popularity,0,25)"</b>,
   "params": {"keywords": "basketball",
              "lat_long_field": "location",
              "user_latitude": 33.748,
              "user_longitude" -84.391}}
  </pre>

The above query does a few interesting things:
- It constructs a query vector containing four features: BM25 Keywords relevance score (higher is better), geo distance (lower is better), publication date (newer is better), and popularity (higher is better).
- Each of the feature values is scaled between 0 and 25 so that they are all comparable, with the best keyword/geo/publication date/popularity score getting a score of 25, and the worst getting a score close to zero.
- Thus a "perfect score" would add up to 100 (25 + 25 + 25 + 25), and the worst score would be approximately 0
- Since the relative contribution of 25 is specified as part of the query for each feature, we can easily change the weights of any feature on the fly to give preference to certain features in the final relevance calculation.

With the above query, we have thus fully taken the relevance function into our own hand by modeling our relevance features and giving them weights. While this is very powerful, it still requires significant manual effort and testing to figure out which features matter for our domain, and what their relative weights should be. In chapter ? we will walk through building Machine-learned Ranking models to automatically make those decisions for us (a process known as "Learning to Rank"). For now, however, our goal was to ensure you understood the mechanics for modeling features in our query vectors, and controlling their weights. 

If you'd like to learn more about how to utilize function queries, I recommend reviewing chapter 7 of [Solr in Action](http://solrinaction.com), "Complex Ranking Functions, for a much fuller exposition. For a full list of available function queries in Solr, you can also check out the [Solr Reference Guide](https://lucene.apache.org/solr/guide/8_3/function-queries.html).

## Matching vs. Ranking
Thus far, we've only really spoken of queries as feature vectors, and we've only discussed relevance ranking as a process of calculating and adding up scores for each each feature (keyword or function) in the query. This may seem a bit strange, since most search books start with coverage of matching keywords in the search engine's inverted index and filter result sets well before discussing relevance.

We've delayed the discussion of "filtering" results until this point on purpose, however, in order to frame what search engines do as two explicitly different actions:
1) Matching: Filtering results to a known set of possible answers
2) Ranking: Ordering all of the possible answers by relevance

In reality, we can completely skip step 1 (filtering) and we'd still see the exact same results on page one (and for many pages), since the most relevant results show up first. If you think back to chapter 2, we even saw some vector scoring calculations (comparing feature vectors for food items - i.e. "apple juice" vs. "donut") that were unable to filter results and had to try to score every document to determine which ones to return based upon relevance.

So if the initial Matching phase is really optional, then why do it at all? The simple answer is that it is primarily a performance optimization. Instead of iterating through every single document and calculating a relevance score, by filtering the initial result set to a smaller set of documents which are logical matches, we can significantly speed up both our relevance calculations and the overall response time of our search engine.

Of course, there are some additional benefits to filtering our results sets, in that the total document count is reduced and we can provide analytics (facets) on the set of logically-matching documents in order to help the user further explore and refind their results set.

But when thinking about constructing relevance functions, like we did in the last section, the idea of filtering and scoring can often get mixed up, particularly since Solr itself mixes concerns in the query parameter.

### Separating Concerns: Filtering vs. Scoring
Solr has two primary ways to control filtering and scoring, the "query" (`q` parameter) and the "filters" (`fq` parameters). Consider the following request:
<pre>
{"query": "the cat in the hat",
 "query_fields": ["description"],
 "filters": [("category", "books"), ("audience", "kid")],
 "query_parser": "edismax",
 "min_match": "100%"}

{"query": "the cat in the hat",
 "params": {"fq": ["category:books", "audience:kid"],
            "qf": ["description"],
            "mm": "100%",
            "defType": "edismax"}}
</pre>

In this query, Solr is being instructed to filter the possible result set down to only documents with a value of `books` in the `category` field and also a value of `kid` in the `audience` field. In addition to those filters, however, the query itself also acts as a filter, so the result set gets further filtered down to only documents also containing (100%) of the values `the`, `cat`, `in`, `the`, and `hat` in the `description` field.

The logical difference between the `q` and `fq` parameters is that the `fq` only acts as a filter, whereas the `q` acts as BOTH a filter and feature vector for relevance ranking. This is useful default behavior for queries, but mixing the concerns of filtering and scoring in the same `q` parameter can be suboptimal, especially if we're simply trying to manipulate the relevance calculation and not arbitrarily removing results from our document set.

There are a few ways to address this:
1. Model the "q" parameter as a function (functions only count toward relevance and do not filter) 
  <pre>
  {<b>"q": '{!func}query("{!type=edismax qf=description mm=100% v=$user_query}")'</b>,
   "params": {"fq": "{!cache=false v=$user_query}",
              "user_query": "the cat in the hat"}}
  </pre>

2. Make your query match all documents (no filtering or scoring) and apply a Boost Query (`bq`) parameter to incluence relevance without scoring 
  <pre>
  {<b>"q": "*:*"</b>,
   "params": {"fq": "{!cache=false v=$user_query}",
              <b>"bq": "{!type=edismax qf=description mm=100% v=$user_quer}"</b>,
              "user_query": "the cat in the hat"}}
  </pre>

Both of these approaches are logically equivalent, but we'll go with option 2 throughout this book since it is a bit cleaner to use the dedicated `bq` parameter which was designed to contribute toward the relevance calculation without filtering. [BIG TODO: Verify that multiplicative boosts still work when specifying a bq= and a boost= using edismax.

## Multiplicative vs. Additive Boosting
One last topic to address concerning how we control our relevance functions is the idea of using multiplicative vs. additive boosting of relevance features.

In all of our examples to this point, we have added multiple features together into our query vector to contribute to the score. For example, the following queries will all yield equivalent relevance calculations assuming they are all filtered down to the same result set (i.e. `fq=the cat in the hat`):
- <pre>{"query": "the cat in the hat"}</pre>
- <pre>{"query": '{!func}query("the") {!func}query("cat") {!func}query("in") {!func}query("the") {!func}query("hat")'}</pre>
- <pre>{"query": "the cat", "params": {"bq": "in the hat"}}</pre>
- <pre>{"query": "*:*", "params": {"bq": "the cat in the hat"}}</pre>
- <pre>{"query": '{!func}query("the cat in the hat")'}</pre>

This kind of relevance is known as Additive Boosting, and maps well to our concept of a query as nothing more than a vector of features in the query that need to have their similarity compared across documents.

In many cases, however, we are likely to want to specify Multiplicative boosts as part of our relevance calculations. Instead of adding additional features to our vector, multiplicative boosts increase the relevance of an entire document by multiplying the document's score by some multiplier.

For example, if we wanted to query for `the cat in the hat`, but wanted the popularity of documents (those with a higher number in the `popularity` field) to have a less constrained effect, we can't easily do this by just adding another feature into our query vector - at least not without modifying the weights of all the other features, plus any additional normalization that may be applied by the BM25 ranking function. If we wanted to apply multiple boosts like this (for example, boosting both on popularity AND on publication date), then the option of modeling this as an additive boost becomes unreasonably complex and harder to control.

In Figure X, we were able to successfully utilize additive boosting by explicitly constraining the minumum and maximum values each feature in our query vector so that each feature provided a known contribution to the overall relevance function.

Multiplicative boosting enables boosts to "pile up" upon each other, however, because each of the boosts is multiplied agains the overall relevance score for the document, resulting in a much fuzzier match and preventing the need for the kind of tight constraints we had to supply for our additive boost example.

To supply a multiplicative boost, you can either use the `{!boost}` query parser in your query vector or, if you are using the `edismax` query parser, the simplified `boost` query param. For example, to multiple a document's relevance score by ten times the value in the popularity field, you would do either: 
  <pre>
  {"query": "the cat in the hat",
   "params": {"defType": "edismax",
              <b>"boost": "mul(popularity,10)"</b>}}</pre>
OR
  <pre>
  {"query": "<b>{!boost b=mul(popularity,10)}</b>the cat in the hat"}
  </pre>

In general, multiplicative boosts enable you greater flexibility to combine different relevance features without having to explicitly pre-define and exact relevance formula accounting for every potential contributing factor. On the other hand, this flexibility can lead to unexpected consequences if the multiplicative boosts values for particular features get too high and overshadow other features. Additive boosts can be a pain to manage, because you have to explicitly scale them so that they can be combined together and maintain a predictible contribution to the overall score, but once you've done this you maintain tight control over the relevance scoring calculation and range of scores.

Both additive and multiplicative boosts can be useful in different scenarios, so it's best to consider the problem at hand and experiment with what gets you the best results.

Up next: Chapter 4 - [Crowdsourced Relevance](../ch04/1.setting-up-the-retrotech-dataset.ipynb)